# Generating object masks from input prompts with SAM

This notebook shows how to generate object masks from input prompts with the Segment Anything Model (SAM). 

Make sure you use GPU runtime for this notebook. 

## Install dependencies

Uncomment and run the following cell to install the required dependencies.

In [ ]:
# %pip install segment-geospatial

In [2]:
import os
import leafmap
from samgeo import SamGeo, tms_to_geotiff

## Create an interactive map

In [22]:
m = leafmap.Map(center=[-6.054643555224412, -50.18255000697929], zoom=12, height="600px")
m.add_basemap("satellite")
m

Map(center=[-6.054643555224412, -50.18255000697929], controls=(ZoomControl(options=['position', 'zoom_in_text'…

## Download a sample image

Pan and zoom the map to select the area of interest. Use the draw tools to draw a polygon or rectangle on the map

In [23]:
if m.user_roi is not None:
    bbox = m.user_roi_bounds()
else:
    bbox = [-50.3658, -6.1518, -50.0311, -5.9716]
bbox

[-50.4595, -6.2796, -49.8525, -5.8767]

In [24]:
image = "data/satellite.tif"
tms_to_geotiff(output=image, bbox=bbox, zoom=12, source="Satellite", overwrite=True)

Downloaded image 01/48
Downloaded image 02/48
Downloaded image 03/48
Downloaded image 04/48
Downloaded image 05/48
Downloaded image 06/48
Downloaded image 07/48
Downloaded image 08/48
Downloaded image 09/48
Downloaded image 10/48
Downloaded image 11/48
Downloaded image 12/48
Downloaded image 13/48
Downloaded image 14/48
Downloaded image 15/48
Downloaded image 16/48
Downloaded image 17/48
Downloaded image 18/48
Downloaded image 19/48
Downloaded image 20/48
Downloaded image 21/48
Downloaded image 22/48
Downloaded image 23/48
Downloaded image 24/48
Downloaded image 25/48
Downloaded image 26/48
Downloaded image 27/48
Downloaded image 28/48
Downloaded image 29/48
Downloaded image 30/48
Downloaded image 31/48
Downloaded image 32/48
Downloaded image 33/48
Downloaded image 34/48
Downloaded image 35/48
Downloaded image 36/48
Downloaded image 37/48
Downloaded image 38/48
Downloaded image 39/48
Downloaded image 40/48
Downloaded image 41/48
Downloaded image 42/48
Downloaded image 43/48
Downloaded 

You can also use your own image. Uncomment and run the following cell to use your own image.

In [25]:
# image = '/path/to/your/own/image.tif'

Display the downloaded image on the map.

In [26]:
m.layers[-1].visible = False
m.add_raster(image, layer_name="Image")
m

Map(bottom=135865.0, center=[-6.158774183267121, -49.9421364068985], controls=(ZoomControl(options=['position'…

## Initialize SAM class

Specify the file path to the model checkpoint. If it is not specified, the model will to downloaded to the working directory.

Set `automatic=False` to disable the `SamAutomaticMaskGenerator` and enable the `SamPredictor`.

In [27]:
sam = SamGeo(
    model_type="vit_h",
    automatic=False,
    sam_kwargs=None,
)

Specify the image to segment. 

In [28]:
sam.set_image(image)

## Image segmentation with input points

A single point can be used to segment an object. The point can be specified as a tuple of (x, y), such as (col, row) or (lon, lat). The points can also be specified as a file path to a vector dataset. For non (col, row) input points, specify the `point_crs` parameter, which will automatically transform the points to the image column and row coordinates.

Try a single point input:

In [13]:
point_coords = [[-50.18255000697929, -6.054643555224412]]
sam.predict(point_coords, point_labels=1, point_crs="EPSG:4326", output="data/mask1.tif")
m.add_raster("data/mask1.tif", layer_name="Mask1", nodata=0, cmap="Blues", opacity=1)
m

Map(bottom=271434.0, center=[-6.161035564149074, -50.15052795410157], controls=(ZoomControl(options=['position…

Try multiple points input:

In [14]:
point_coords = [ [-50.29936048403308, -6.103175551795105] ,[-50.18255000697929, -6.054643555224412]]
sam.predict(point_coords, point_labels=1, point_crs="EPSG:4326", output="data/mask2.tif")
m.add_raster("data/mask2.tif", layer_name="Mask2", nodata=0, cmap="Greens", opacity=1)
m

Map(bottom=1084504.0, center=[-6.104370144775546, -50.26262283325196], controls=(ZoomControl(options=['positio…

## Interactive segmentation

Display the interactive map and use the marker tool to draw points on the map. Then click on the `Segment` button to segment the objects. The results will be added to the map automatically. Click on the `Reset` button to clear the points and the results.

In [15]:
m = sam.show_map()
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

# Box Prompts

In [32]:
boxes = [
    [-50.2216, -6.1182, -50.1028, -6.0186],
    [-50.3294, -6.1244, -50.2697, -6.0725],
]

In [33]:
sam.predict(boxes=boxes, point_crs="EPSG:4326", output="mask.tif", dtype="uint8")

In [34]:
m.add_raster("mask.tif", cmap="viridis", nodata=0, layer_name="Mask")
m

Map(bottom=135806.0, center=[-6.077742151516975, -50.15533447265625], controls=(ZoomControl(options=['position…